In [ ]:
import pickle
import pandas as pd

In [78]:
ticker = "ES"
interval = 30           # Assumed to be either 5, 10, 15, 30, 50 minutes

with open("data/futures_ticks.pkl", "rb") as f:
    ticks = pickle.load(f)[ticker]

ticker += "_temp"       # Temp!

# Pull historical data on defined future ticker
futures_df = pd.read_csv(f"./data/S&P500 Mini/{ticker}.csv", index_col='t', parse_dates=True)

# Resample dataframe to defined holding period
resampled_o = futures_df['o'].resample(f"{interval}min").first()
resampled_h = futures_df['h'].resample(f"{interval}min").max()
resampled_l = futures_df['l'].resample(f"{interval}min").min()
resampled_c = futures_df['c'].resample(f"{interval}min").last()
resampled_v = futures_df['v'].resample(f"{interval}min").sum()

main_df = pd.DataFrame(
    {
        'o': resampled_o,
        'h': resampled_h,
        'l': resampled_l,
        'c': resampled_c,
        'v': resampled_v,
    }
)

main_df["fwd_r_u"] = ((main_df["h"] - main_df["o"]) / ticks).shift(-1)
main_df["fwd_r_d"] = ((main_df["o"] - main_df["l"]) / ticks).shift(-1)
main_df["fwd_r_o"] = main_df["fwd_r_u"] + main_df["fwd_r_d"]

main_df

,o,h,l,c,v,fwd_r_u,fwd_r_d,fwd_r_o
t,,,,,,,,
2019-12-15 18:00:00,3179.50,3180.50,3177.00,3178.50,5571,6.0,1.0,7.0
2019-12-15 18:30:00,3178.50,3180.00,3178.25,3178.50,2996,9.0,1.0,10.0
2019-12-15 19:00:00,3178.50,3180.75,3178.25,3179.25,3417,7.0,2.0,9.0
2019-12-15 19:30:00,3179.25,3181.00,3178.75,3180.00,2393,5.0,1.0,6.0
2019-12-15 20:00:00,3180.00,3181.25,3179.75,3180.75,3291,8.0,2.0,10.0
...,...,...,...,...,...,...,...,...
2020-06-17 21:30:00,3090.00,3090.00,3075.25,3085.00,3605,1.0,28.0,29.0
2020-06-17 22:00:00,3084.75,3085.00,3077.75,3082.00,1543,19.0,23.0,42.0
2020-06-17 22:30:00,3081.75,3086.50,3076.00,3083.75,1422,49.0,5.0,54.0


In [ ]:
# Predictor generation
main_df

,o,h,l,c,v,r_u,r_d,r_o
t,,,,,,,,
2019-12-15 18:00:00,3179.50,3180.50,3177.00,3178.50,5571,4.0,10.0,14.0
2019-12-15 18:30:00,3178.50,3180.00,3178.25,3178.50,2996,6.0,1.0,7.0
2019-12-15 19:00:00,3178.50,3180.75,3178.25,3179.25,3417,9.0,1.0,10.0
2019-12-15 19:30:00,3179.25,3181.00,3178.75,3180.00,2393,7.0,2.0,9.0
2019-12-15 20:00:00,3180.00,3181.25,3179.75,3180.75,3291,5.0,1.0,6.0
...,...,...,...,...,...,...,...,...
2020-06-17 21:30:00,3090.00,3090.00,3075.25,3085.00,3605,0.0,59.0,59.0
2020-06-17 22:00:00,3084.75,3085.00,3077.75,3082.00,1543,1.0,28.0,29.0
2020-06-17 22:30:00,3081.75,3086.50,3076.00,3083.75,1422,19.0,23.0,42.0


To-do:
- Calculate rolling volatility and volume: EMA
- Stratifying rolling data: percentiles
- Logging range of each stratum, building empirical PDF for each subgroup

Notes:
- Cross-check with Hannah: if minimum tick moves match what I found
- Extra: Using a Poisson process to model hit rate to see if limit order slicing is any good